# Training a cNN to detect roadsigns
In order to process roadsigns in the autonomous car of the Freie Universität, we want to train a convolutional (deep) neural network.

This network is supposed to distinguish between different classes of signs (stop, attention, train crossing etc) and the final model will then be integrated to the autonomos ROS structure.

This notebook shall download the dataset, read it in and then train the classifier. Afterwards, a validation of the training procedure will be done.

In [26]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import urllib2, cStringIO, zipfile
import csv
import os
import random

## Set global parameters such as image size or channels

**Options include:**
* Size of the images
* Which color channels are used (RGB or YUV)
* How many new images are artificially added

In [8]:
img_size = (32, 32)

## Download the dataset

In [9]:
url = 'http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Training_Images.zip'

if not os.path.exists('GTSRB/Final_Training/Images'):
    try:
        remotezip = urllib2.urlopen(url)
        zipinmemory = cStringIO.StringIO(remotezip.read())
        zip = zipfile.ZipFile(zipinmemory)
        zip.extractall('.')
    except urllib2.HTTPError:
        pass

## Read the data in and scale it to a fixed resolution

In [17]:
def getSingleImage(path):
    img = Image.open(path) # the 1th column is the filename
    img_resized = img.resize((img_size[0], img_size[1]), Image.LINEAR)
    img_resized = img_resized.convert('YCbCr')
    Y_channel = img_resized.split()[0]
    del img, img_resized
    return np.array(Y_channel.getdata(), dtype=np.float32).reshape(img_size[0], img_size[1], 1)

def readTrafficSigns(rootpath):
    '''Reads traffic sign data for German Traffic Sign Recognition Benchmark.

    Arguments: path to the traffic sign data, for example './GTSRB/Training'
    Returns:   list of images, list of corresponding labels'''

    training_images = [] # images
    training_labels = [] # corresponding labels
    testing_images = []
    testing_labels = []
    
    test_class = random.randrange(0, 42)
    print "Using class " + str(test_class) + " as test set!"

    # loop over all 43 classes
    for c in range(0,43):
        prefix = rootpath + '/' + format(c, '05d') + '/' # subdirectory for class
        gtFile = open(prefix + 'GT-'+ format(c, '05d') + '.csv') # annotations file
        gtReader = csv.reader(gtFile, delimiter=';') # csv parser for annotations file
        gtReader.next() # skip header

        # loop over all images in current annotations file
        for row in gtReader:
            np_img = getSingleImage(prefix + row[0]) # the 1th column is the filename
            if test_class == c:
                testing_images.append(np_img)
                testing_labels.append(row[7]) # the 8th column is the label
            else:
                training_images.append(np_img)
                training_labels.append(row[7]) 
        gtFile.close()
        print "Loaded images from class " + str(c)
    return (training_images, training_labels, testing_images, testing_labels)

In [18]:
trainImg, trainLabels, testImg, testLabels = readTrafficSigns('GTSRB/Final_Training/Images')

Using class 24 as test set!
Loaded images from class 0
Loaded images from class 1
Loaded images from class 2
Loaded images from class 3
Loaded images from class 4
Loaded images from class 5
Loaded images from class 6
Loaded images from class 7
Loaded images from class 8
Loaded images from class 9
Loaded images from class 10
Loaded images from class 11
Loaded images from class 12
Loaded images from class 13
Loaded images from class 14
Loaded images from class 15
Loaded images from class 16
Loaded images from class 17
Loaded images from class 18
Loaded images from class 19
Loaded images from class 20
Loaded images from class 21
Loaded images from class 22
Loaded images from class 23
Loaded images from class 24
Loaded images from class 25
Loaded images from class 26
Loaded images from class 27
Loaded images from class 28
Loaded images from class 29
Loaded images from class 30
Loaded images from class 31
Loaded images from class 32
Loaded images from class 33
Loaded images from class 34
Lo

In [19]:
print "Number of training images: " + str(len(trainImg))
print "Number of training labels: " + str(len(trainLabels))
print "Number of testing images: " + str(len(testImg))
print "Number of testing labels: " + str(len(testLabels))

Number of training images: 38939
Number of training labels: 38939
Number of testing images: 270
Number of testing labels: 270


## Permute the training data randomly and make it a big matrix

In [22]:
permutation = np.random.permutation(len(trainImg))
train_set = np.array([trainImg[idx] for idx in permutation], dtype=np.float32)
train_labels = [trainLabels[idx] for idx in permutation]
test_set = np.array(testImg, dtype=np.float32)
test_labels = testLabels

## Transform labels to one-hot-vectors and make it a matrix

In [23]:
train_labels_oh = []
number_of_classes = 43
for label in train_labels:
    new_label = np.zeros(number_of_classes)
    new_label[int(label)] = 1
    train_labels_oh.append(new_label)
train_labels = np.array(train_labels_oh, dtype=np.float32)

test_labels_oh = []
for label in test_labels:
    new_label = np.zeros(number_of_classes)
    new_label[int(label)] = 1
    test_labels_oh.append(new_label)
test_labels = np.array(test_labels_oh, dtype=np.float32)

## Show some of the images

In [ ]:
%matplotlib inline
random_images = [random.randrange(train_set.shape[0]) for i in xrange(10)]
print random_images
fig = plt.figure()
for i in xrange(10):
    fig.add_subplot(2, 5, i+1)
    plt.imshow(train_set[random_images[i]])
plt.show()

## Pickle the images to a file that can easily be loaded